<a href="https://colab.research.google.com/github/SLCFLAB/Data-Science-Python/blob/main/Day%2011/11_2.CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

출처: https://github.com/SLCFLAB/Fintech2022, https://github.com/Harry24k/Pytorch-Basic


GPU 사용하는 방법: 런타임 -> 런타임 유형 변경 -> GPU

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import random
import os

import matplotlib.pyplot as plt
%matplotlib inline

## Load data

In [2]:
train_data = dsets.MNIST(root='data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

test_data = dsets.MNIST(root='data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
batch_size = 100 # 한번에 할 연산량

train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv_layer = nn.Sequential( ## To DO: 주석으로 달아둔 shape과 다음 layer를 참고해 ?를 채워주세요
                                #input: 100(batch) * 28 * 28
            nn.Conv2d(1, 16, ?), #[100, ? ,24,24]
            nn.ReLU(),
            nn.Conv2d(16, ?, 5),#[100,32,?,?]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),# 100,32,?,?
            nn.Conv2d(32, ?, 5),# 100,64,6,6
            nn.ReLU(),
            nn.MaxPool2d(?, ?) # 100,64,3,3 -> 여기서 batch 빼고 나머지 64*3*3
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100), #여기서 64*3*3 -> 100
            nn.ReLU(),
            nn.Linear(100, 10)#100 -> 10
        )       
        
    def forward(self,x):
        out = self.conv_layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out

In [ ]:
model = CNN().cuda() ## GPU 사용

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Train model

In [ ]:
num_epochs = 3

In [ ]:
for epoch in range(num_epochs):

    total_batch = len(train_data) // batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):

        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/3], lter [300/600], Loss: 2.2847
Epoch [1/3], lter [600/600], Loss: 1.9827
Epoch [2/3], lter [300/600], Loss: 0.4843
Epoch [2/3], lter [600/600], Loss: 0.2493
Epoch [3/3], lter [300/600], Loss: 0.1727
Epoch [3/3], lter [600/600], Loss: 0.1895


## Test model

In [ ]:
correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 94.590000 %
